In [1]:
import climt
from climt._core.initialization import HybridSigmaPressureDiagnosticComponent
from gfs_dynamical_core import GFSDynamicalCore
from sympl import (
    PlotFunctionMonitor, NetCDFMonitor, set_constant,
    TimeDifferencingWrapper, UpdateFrequencyWrapper,
    get_constant, reset_constants, DataArray, DiagnosticComponent,
    combine_component_properties
)
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import xarray as xr
from xarray import DataArray
import xesmf as xe
from scipy.interpolate import CubicSpline
import pkg_resources

In [2]:

# def plot_function(fig, state):

#     ax = fig.add_subplot(2, 2, 1)
#     state['specific_humidity'].mean(
#         dim='lon').plot.contourf(
#             ax=ax, levels=16, robust=True)
#     ax.set_title('Specific Humidity')

#     ax = fig.add_subplot(2, 2, 3)
#     state['eastward_wind'].mean(dim='lon').plot.contourf(
#         ax=ax, levels=16, robust=True)
#     ax.set_title('Eastward Wind')

#     ax = fig.add_subplot(2, 2, 2)
#     state['northward_wind'].transpose().mean(
#         dim='lon').plot.contourf(
#         ax=ax, levels=16, robust=True)
#     ax.set_title('Northward Wind')

#     ax = fig.add_subplot(2, 2, 4)
#     state['air_temperature'].mean(dim='lon').plot.contourf(
#         ax=ax, levels=16)
#     ax.set_title('Temperature')

#     fig.tight_layout()

# # Create plotting object
# fields_to_store = ['air_temperature', 'northward_wind', 'eastward_wind',
#                     'divergence_of_wind', 'atmosphere_relative_vorticity',
#                     'specific_humidity', 'surface_specific_humidity',
#                     'surface_temperature', 'surface_geopotential']
                   
# monitor = PlotFunctionMonitor(plot_function)
# netcdf_monitor = NetCDFMonitor('wip_notbook_monitor.nc',
#                                write_on_store=True,
#                                store_names=fields_to_store)

def plot_function(fig, state):

    ax = fig.add_subplot(1, 1, 1)
    CS = ax.contourf(state['longitude'], state['latitude'],
                     state['surface_air_pressure'].to_units('mbar'))
    plt.colorbar(CS)
    ax.set_title('Surface Pressure at: '+str(state['time']))

    plt.show()

monitor = PlotFunctionMonitor(plot_function)

<Figure size 640x480 with 0 Axes>

In [28]:

era5_pressure_data = xr.open_dataset('/run/media/adamh/X6/test_levels/0101180000_allhPa.nc').isel(time=0)
era5_surface_data = xr.open_dataset('/run/media/adamh/X6/test_levels/1Jan18_surface.nc').isel(time=0)

### air_pressure = era5_pressure_data['pressure'].values  # 3D array: [levels, lat, lon]

latitude = era5_pressure_data['latitude']
longitude = era5_pressure_data['longitude']
levels = era5_pressure_data['level']

# 3D array: [level, lat, lon]
divergence_of_wind = era5_pressure_data['d']
geopotential = era5_pressure_data['z']
specific_humidity = era5_pressure_data['q']
air_temperature = era5_pressure_data['t']     
eastward_wind = era5_pressure_data['u']    
northward_wind = era5_pressure_data['v']  
relative_vorticity = era5_pressure_data['vo']   

# 2D array: [lat, lon]
dewpoint_temperature = era5_surface_data['d2m']
surface_temperature = era5_surface_data['t2m']
surface_geopotential = era5_surface_data['z']
surface_pressure = era5_surface_data['sp'] 

era5_data = [divergence_of_wind, geopotential, specific_humidity,
             air_temperature, eastward_wind, northward_wind,
             relative_vorticity, dewpoint_temperature, 
             surface_temperature, surface_geopotential,
             surface_pressure]

In [29]:
climt_grid = np.load('/run/media/adamh/X6/test_levels/climt_lat_lon.npz')
climt_latitude_standard = climt_grid['latitude']
climt_longitude_standard = climt_grid['longitude']

# Create a new grid for the regridding
climt_grid = xr.Dataset(
    {
        'lat': (['lat'], climt_latitude_standard),
        'lon': (['lon'], climt_longitude_standard),
    }
)

# Function to regrid data
def xr_regrid(ds):

    # Create the regridder
    regridder = xe.Regridder(
        ds,                         # original data
        climt_grid,                 # new grid
        method='conservative'           # choose your method ('bilinear', 'nearest_s2d', etc.)
    )

    # Perform the regridding
    regridding = regridder(ds)

    if regridding.ndim == 3:
        return regridding.isel(level=slice(None, None, -1))
    else:
        return regridding

divergence_of_wind_regrid = xr_regrid(divergence_of_wind)
geopotential_regrid = xr_regrid(geopotential)
specific_humidity_regrid = xr_regrid(specific_humidity)
air_temperature_regrid = xr_regrid(air_temperature)
eastward_wind_regrid = xr_regrid(eastward_wind)
northward_wind_regrid = xr_regrid(northward_wind)
relative_vorticity_regrid = xr_regrid(relative_vorticity)
dewpoint_temperature_regrid = xr_regrid(dewpoint_temperature)
surface_temperature_regrid = xr_regrid(surface_temperature)
surface_geopotential_regrid = xr_regrid(surface_geopotential)
surface_pressure_regrid = xr_regrid(surface_pressure)

/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xesmf/backend.py:56: UserWa

In [33]:
# Combine DataArrays into a Dataset
era5_data = xr.Dataset({
    'divergence_of_wind_regrid': divergence_of_wind_regrid,
    'geopotential_regrid': geopotential_regrid,
    'specific_humidity_regrid': specific_humidity_regrid,
    'air_temperature_regrid': air_temperature_regrid,
    'eastward_wind_regrid': eastward_wind_regrid,
    'northward_wind_regrid': northward_wind_regrid,
    'relative_vorticity_regrid': relative_vorticity_regrid,
    'dewpoint_temperature_regrid': dewpoint_temperature_regrid,
    'surface_temperature_regrid': surface_temperature_regrid,
    'surface_geopotential_regrid': surface_geopotential_regrid,
    'surface_pressure_regrid': surface_pressure_regrid
})

# Save the Dataset to a NetCDF file
era5_data.to_netcdf('/run/media/adamh/X6/test_levels/era5_regrid_0101180000.nc')

In [3]:
# Load the Dataset from the NetCDF file
loaded_era5_data = xr.open_dataset('/run/media/adamh/X6/test_levels/era5_regrid_0101180000.nc')

# Access the DataArrays by their new names
divergence_of_wind_regrid = loaded_era5_data['divergence_of_wind_regrid']
geopotential_regrid = loaded_era5_data['geopotential_regrid']
specific_humidity_regrid = loaded_era5_data['specific_humidity_regrid']
air_temperature_regrid = loaded_era5_data['air_temperature_regrid']
eastward_wind_regrid = loaded_era5_data['eastward_wind_regrid']
northward_wind_regrid = loaded_era5_data['northward_wind_regrid']
relative_vorticity_regrid = loaded_era5_data['relative_vorticity_regrid']
dewpoint_temperature_regrid = loaded_era5_data['dewpoint_temperature_regrid']
surface_temperature_regrid = loaded_era5_data['surface_temperature_regrid']
surface_geopotential_regrid = loaded_era5_data['surface_geopotential_regrid']
surface_pressure_regrid = loaded_era5_data['surface_pressure_regrid']

/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [4]:
surface_pressure_regrid_mean = np.mean(surface_pressure_regrid.values)
top_of_era5_pressure_in_Pa = (air_temperature_regrid.coords['level'].values.min())*1000
set_constant('reference_air_pressure', value=np.mean(surface_pressure_regrid_mean), units='Pa')
set_constant('top_of_model_pressure', value=top_of_era5_pressure_in_Pa, units='Pa')

grid = climt.get_grid(nx=128, ny=64, nz=37, n_ice_interface_levels=None,
                      p_surf_in_Pa=surface_pressure_regrid_mean, 
                      p_toa_in_Pa=top_of_era5_pressure_in_Pa,
                      custom_surface_pressure=surface_pressure_regrid)             

                      
dycore = GFSDynamicalCore() 
my_state = climt.get_default_state([dycore], grid_state=grid)

dcmip = climt.DcmipInitialConditions(add_perturbation=False)
out = dcmip(my_state)

In [5]:
timestep = timedelta(minutes=10)

print(my_state['air_pressure'])

for i in range(1):
    diag, output = dycore(my_state, timestep)
    monitor.store(my_state)
    my_state.update(output)
    my_state['time'] += timestep

print(air_temperature_regrid)

<xarray.DataArray (mid_levels: 37, lat: 64, lon: 128)> Size: 2MB
array([[[100627.94051916, 100626.24411897, 100625.31056344, ...,
         100627.54946983, 100621.88197422, 100623.39579367],
        [100905.79683136, 100901.92749341, 100884.6397517 , ...,
         100953.32482753, 100933.21709898, 100927.75090019],
        [100448.0869816 , 100326.4778062 , 100297.40270184, ...,
         100925.52867562, 100774.70802907, 100608.65643493],
        ...,
        [ 74381.98568722,  73450.19682669,  72674.29060415, ...,
          76354.87904449,  75848.66449428,  75238.73646044],
        [ 71595.33548647,  71380.06077714,  71211.36231332, ...,
          72459.07602251,  72132.44310921,  71844.56983695],
        [ 70920.60244869,  70744.19667159,  70563.17836835, ...,
          71370.766185  ,  71222.39635606,  71070.5294428 ]],

       [[100051.66729893, 100049.98200453, 100049.05456072, ...,
         100051.27880967, 100045.64841743, 100047.15232635],
        [100327.70457105, 100323.86056

/tmp/ipykernel_6522/3168687931.py:43: UserWarning: Adding colorbar to a different Figure <Figure size 640x480 with 2 Axes> than <Figure size 640x480 with 0 Axes> which fig.colorbar is called on.
  plt.colorbar(CS)


<Figure size 640x480 with 0 Axes>

<xarray.DataArray 'air_temperature_regrid' (level: 37, lat: 64, lon: 128)> Size: 2MB
[303104 values with dtype=float64]
Coordinates:
  * level    (level) int32 148B 1000 975 950 925 900 875 850 ... 20 10 7 5 3 2 1
    time     datetime64[ns] 8B ...
  * lat      (lat) float64 512B 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
  * lon      (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
Attributes:
    regrid_method:  conservative


In [41]:
for key in my_state.keys():
    print(key)

atmosphere_hybrid_sigma_pressure_a_coordinate_on_interface_levels
atmosphere_hybrid_sigma_pressure_b_coordinate_on_interface_levels
surface_air_pressure
time
air_pressure
air_pressure_on_interface_levels
longitude
latitude
air_temperature
eastward_wind
northward_wind
divergence_of_wind
atmosphere_relative_vorticity
surface_geopotential
specific_humidity
